# Approximate H2 molecule energy near ground state

In this example notebook, we'll walk through calculating the energy of a molecule given an estimate for the ground state using Azure Quantum. We will do that using a simple algorithm where we prepare the quantum register in a state near the ground state of the molecule using the Jordan-Wigner encoding, and evaluate the energy of the molecule by applying the Hamiltonian operator to that state. To be able to run the algorithm on near-term hardware, we will estimate the energy per Hamiltonian term, and then calculate the total estimated energy by adding the values of these terms.

In [ ]:
# First, import qsharp and qsharp.azure to be able to compile and submit the quantum program.
import qsharp
import qsharp.azure
qsharp.reload()
from Microsoft.Quantum.Chemistry.Hamiltonian import GetHamiltonianTermH2, ExpandedCoefficients_

### 1. Load H2 molecule data

The pre-generated Broombridge file included in this sample contains the details on the molecule's Hamiltonian. A Hamiltonian is an operator that calculates the energy of a molecule by acting on a qubit register that represents the molecule's quantum state by Jordan-Wigner encoding. If we prepare the molecule in the ground state, then we know that applying the Hamiltonian will calculate the ground state energy.

In [ ]:
from qdk.chemistry.broombridge import load_and_encode

In [ ]:
encoded_data_h2 = load_and_encode("../data/broombridge/hydrogen_0.2.yaml")

This contains a tuple of the number of qubits, the fermionic Hamiltonian term coefficients and the energy offset.

The index of each list in the second tuple is will be referred to as the "term type".

In [ ]:
num_qubits, fermion_terms, _, energy_offset = encoded_data_h2

### 2. Resource estimation
Before running on hardware, let's first estimate the quantum resources needed for measuring two-electron terms ("term type" = 3)

In [ ]:
GetHamiltonianTermH2.estimate_resources(
    nOp=12
)

We have 11 qubits on the IonQ system so this is feasible to run!

### 3. Running on Azure Quantum

Connect to Azure Quantum

In [ ]:
rid = "/subscriptions/dcdedb1f-5556-4212-89bf-9621f96fd0f2/resourceGroups/AzureQuantum/providers/Microsoft.Quantum/Workspaces/jkingdon-qw" # Enter your workspace's resource ID here
location = "East US" # Enter your workspace's location here, e.g. "West US"

In [ ]:
qsharp.azure.connect(
   resourceId=rid,
   location=location,
   credential="devicecode"
)

In [ ]:
qsharp.azure.target("ionq.simulator")

Run a single term on the IonQ simulator:

In [ ]:
n_op = 17
result = qsharp.azure.execute(GetHamiltonianTermH2, nOp=n_op, shots=1000, jobName=f"Hamiltonian term {n_op}")

In [ ]:
result

### 4. Loop over all Hamiltonian terms

Each Hamiltonian term has a term type, measurement operator and coefficient. To simplify this sample, we will move the coefficients into a flat list. Each nonzero coefficient will render a separate job that we send to Azure Quantum. To calculate the energy for that term, we map result from the job onto a -1 to 1 axis and multiply the value by the corresponding coefficient.

In [ ]:
coeffs = []
for term_type, terms in enumerate(fermion_terms):
    for (qubits, coeff) in terms:
        coeffs += ExpandedCoefficients_(coeff=coeff, termType=term_type)
coeffs

There are in total 18 coefficients corresponding to each combination of Hamiltonian term and Jordan-Wigner measurement operator. Start a job for each term that has a nonzero coefficient:

In [ ]:
jobs = [
    qsharp.azure.submit(GetHamiltonianTermH2, nOp=n_op, shots=1000, jobName=f"Hamiltonian term {n_op}") 
    for n_op, coeff in enumerate(coeffs) if coeff != 0.0
]

In [ ]:
results = [qsharp.azure.output(j.id) for j in jobs]
results

### 5. Estimate energy

In [ ]:
def calc_energy(coeffs, results):
    return sum([(2. * res.get("0", 0.0) - 1.) * coeff for coeff, res in zip(coeffs, results)]) + energy_offset

In [ ]:
calc_energy(coeffs, results)

Now we are ready to run on hardware! Switch to the QPU.

In [ ]:
qsharp.azure.target("ionq.qpu")

In [ ]:
jobs_hw = [
    qsharp.azure.submit(GetHamiltonianTermH2, nOp=n_op, shots=1000, jobName=f"Hamiltonian term {n_op}") 
    for n_op, coeff in enumerate(coeffs) if coeff != 0
]

Check if the jobs succeeded.

In [ ]:
jobs_status = [qsharp.azure.status(j) for j in jobs_hw]
[
    j.status + " | " + j.name + " | " + j.target + " | " + j.creation_time + " | " + j.end_execution_time 
    for j in jobs_status
]

Get results and estimate energy

In [ ]:
results = [qsharp.azure.output(j.id) for j in jobs_hw]
results

In [ ]:
calc_energy(coeffs, results)